# Probabilistic models

* In order to use any of the previous VLCs (Variable Length Codecs), a probabilistic model is needed.

## 1. Static models

* Static models are the simplest ones because they suppose that the probabilities of the symbols
  remain constant through the compression/decompression.
* In this case, variable-length code-words can be precomputed.
* Static models are very common in codecs such
  as JPEG and MPEG (audio and video).

## 2. Adaptive models

* The probabilities of the symbols are computed in run-time.
* In general, the compression ratios that adaptive models
  get are better than static model's ones, because the
  probabilities of the symbols are localy computed
  (think of the sequence `aaaaaaaaaaaaaabbbbbbbbbbbbbbb`).

### Encoding

1. Asign the same probability to all the symbols of the source alphabet.
2. While the input if not exhausted:
    1. Encode the next symbol.
    2. Update (increase) its probability.

### Example
TO-DO

### Decoding

1. Identical to the step 1 of the encoder.
2. While the input is not exhausted:
    1. Decode the next symbol.
    2. Identical to the step 2.B of the encoder.

### Example
TO-DO

## 3. Initially empty models

* The smaller the number of symbols used by the model, the higher
  the probabilities, and therefore, the better the compression ratios.
* An initially empty model only stores the $\mathtt{ESC}$(cape) symbol (a
  symbol that it is used by the encoder only when a new symbol is
  found).

### Encoder

1. Set the probability of the $\mathtt{ESC}$ symbol to $1.0$ (and the probability of
   the rest of the symbols to $0.0$).
2. While the input is not exhausted:
    1. $s\leftarrow$ next symbol.
    2. If $s$ has been found before, then:
        1. Output $c(s)$ (encode).
    3. Else:
        1. Output $c(\mathtt{ESC})$.
        2. Output a raw symbol $s$.
    4. Update $p(s)$.

### Example
TO-DO

### Decoder

1. Identical to the step 1 of the encoder.
2. While the input is not exhausted:
    1. $c(s)\leftarrow $ next code-word.
    2. Decode $s$.
    3. If $s=\mathtt{ESC}$, then:
        1. Input a raw symbol $s$.
    4. Update $p(s)$.
    5. Output $s$.

### Example
TO-DO

   
## 4. Models with memory

* In most cases, the probability of a symbol depends on its
  neighborhood (context).
* The higher the memory of the model (the context), the higher the
  accuracy of the predictions (probabilities), and therefore, the
  lower the length of the code-words [[J. Cleary & I. Whitten]](https://scholar.google.es/scholar?hl=es&as_sdt=0%2C5&q=Data+Compression+using+Adaptive+Coding+and+Partial+String+Matching&btnG=).
* Let ${\cal C}[i]$ the last $i$ encoded symbols and
  $p(s|{\cal C}[i])$ the probability that the symbol $s$ follows
  the context ${\cal C}[i]$.
* Let $k$ the maximal order of the prediction (i.e. the largest
  number of symbols of ${\cal C}[]$ that are going to be used as the
  actual context). Notice that ${\cal C}[0]=\varnothing$ and the model
  has no memory.
* We suppose that arithmetic coding is used and therefore, when we
  input or output $c(s)$, we are transmitting $I(s)$ bits of code.
* Let $r$ the size of the source alphabet.

### Encoder

1. Create an empty model for every context $0\le i \le k$.
2. Create an non-empty model for $k=-1$.
3. While the input is not exhausted:
    1. $s\leftarrow$ Input ${\log_2(r)}$ bits.
    2. $i\leftarrow k$ (except for the first symbol, where
       $i\leftarrow 0$).
    3. While $p(s|{\cal C}[i])=0$ (it is the first time that $s$ follows
       ${\cal C}[i]$):
        1. Output $\leftarrow c(\text{ESC}|{\cal C}[i])$.
        2. Update $p(\text{ESC}|{\cal C}[i])$.
        3. Update $p(s|{\cal C}[i])$ (insert $s$ into the ${\cal C}[i]$ context).
        4. $i\leftarrow i-1$.
    4. Output $\leftarrow c(s|{\cal C}[i])$. The symbols that were in
       contexts with order $>i$ must be excluded of the actual (${\cal C}[i]$) context because $s$ is not none of them.
    5. If $i\ge 0$, update $p(s|{\cal C}[i])$.

### Example

* Let $r=256$ the size of the source alphabet.

* The probabilistic model $M[{\cal C}[-1]]$ (for the special context
  ${\cal C}[-1]$) is static, non empty and has an special symbol $\mathtt{EOF}$
  (End Of File) that is going to be used when the compression has
  finished:
  $$M[{\cal C}[-1]]=\{0,1~1,1~\cdots~\mathtt{a},1~\mathtt{b},1~\cdots~255,1~\mathtt{EOF},1\}.$$
  In a pair $(a,b)$, $a$ is the symbol and $b$ is its probability (counts).

* $M[{\cal C}[0]]$ is adaptative and empty:
  $$M[{\cal C}[0]]=\{\text{ESC},1\}.$$

* In this example (for the sake of the simplicity), the maximal
  order of the prediction $k=1$ (we only remember the previous
  symbol). Therefore, there are $r=256$ probabilistic models:
  $$M[{\cal C}[1]]=\{\text{ESC},1\}, 0\le {\cal C}[1]\le r.$$
  
* **Encoding of the first symbol ($\mathtt{a}$)** (see the following figure):

1. [3.A] $s\leftarrow \mathtt{a}$.
2. [3.B] $i\leftarrow 0$ (we don't know the previous symbol).
3. [3.C] $p(\mathtt{a}|{\cal C}[0])=0$ (the context only has the $\mathtt{ESC}$).
4. [3.C.a] Output $\leftarrow c(\mathtt{ESC}|{\cal C}[0])$ (althought
    $l(c(\mathtt{ESC}|{\cal C}[0]))=0$).
5. [3.C.b] Update $p(\mathtt{ESC}|{\cal C}[0])$ (now, the count of ESC is
    2).
6. [3.C.c] Insert $\mathtt{a}$ into
    $M[{\cal C}[0]]=\{\mathtt{ESC},2~\mathtt{a},1\}$.
7. [3.C.d] $i\leftarrow -1$.
8. [3.c] $p(\mathtt{a}|{\cal C}[-1])\neq 0$.
9. [3.d] Output $\leftarrow c(\mathtt{a}|{\cal C}[-1])$ where
    $p(\mathtt{a}|{\cal C}[-1]) = 1/(256+1)$.
    
* **Encoding of the second symbol ($\mathtt{b}$)**:

1. [3.a] $s\leftarrow \mathtt{b}$.
2. [3.b] $i\leftarrow 1$.
3. [3.c] $p(\mathtt{b}|{\cal C}[1])=0$ because ${\cal C}[1]=\mathtt{a}$ and
   $M[\mathtt{a}]=\{\mathtt{ESC},1\}$.
4. [3.c.i] Output $\leftarrow c(\mathtt{ESC}|\mathtt{a})$ (althought
   $l(c(\mathtt{ESC}|\mathtt{a}))=0$).
5. [3.c.ii] Update $p(\mathtt{ESC}|\mathtt{a})$ (now, the count of $\mathtt{ESC}$ is 2).
6. [3.c.iii] Insert $\mathtt{b}$ into $M[\mathtt{a}]=\{\mathtt{ESC},2~ \mathtt{b},1\}$.
7. [3.c.iv] $i\leftarrow 0$.
8. [3.c] $p(\mathtt{b}|{\cal C}[0])=0$ because
   $M[{\cal C}[0]]=\{\mathtt{ESC},2~\texttt{a},1\}$.
9. [3.c.i] Output $\leftarrow c(\mathtt{ESC}|{\cal C}[0])$ where
   $p(\mathtt{ESC}|{\cal C}[0]) = 2/3$.
10. [3.c.ii] Update $p(\mathtt{ESC}|{\cal C}[0])$ (now, the count of ESC is
    3).
11. [3.c.iii] Insert $\mathtt{b}$ into $M[{\cal C}[0]] = \{\mathtt{ESC},3~
    \mathtt{a},1~ \mathtt{b},1\}$.
12. [3.c.iv] $i\leftarrow -1$.
13. [3.c] $p(\mathtt{b}|{\cal C}[-1])\neq 0$.
14. [3.d] Output $\leftarrow c(\mathtt{b}|{\cal C}[-1])$ where
    $p(\mathtt{b}|{\cal C}[-1]) = 1/r$. The symbol $\mathtt{a}$ has been
    excluded in the calculus of the probability of $\mathtt{b}$ because
    $\mathtt{a}\in M[{\cal C}[0]] = \{\mathtt{ESC},3~ \mathtt{a},1~
    \mathtt{b},1\}$.

<img src="data/PPM_example.png" width="800">

### Decoder

1. Equal to the step 1 of the encoder.
2. While the input is not exhausted:
    1. $i\leftarrow k$ (except for the first symbol, where $i\leftarrow 0$).
    2. $s\leftarrow$ next decoded symbol.
    3. While $s=\mathtt{ESC}$:
        1. Update $p(\mathtt{ESC}|{\cal C}[i])$.
        2. $i\leftarrow i-1$.
        3. $s\leftarrow$ next decoded symbol.
    4. Update $p(s|{\cal C}[i])$.
    5. While $i<k$:
        1. $i\leftarrow i+1$.
        2. Update $p(s|{\cal C}[i])$ (insert $s$ into the ${\cal C}[i]$ context).

### Lab
TO-DO